In [1]:
%pip install openai pydantic jinja2 python_dotenv ipywidgets d20

In [2]:
import dis
import os
import re
import tomllib
import functools

from jinja2 import Template
from numpy import diff
from pydantic import BaseModel
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv
from d20 import roll

import ipywidgets as widgets
from IPython.display import display, clear_output

from typing import Literal, Union

from sympy import fu

load_dotenv()

with open("data/example_haunted_house.toml", "rb") as f:
    config = tomllib.load(f)

message_pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
class Message(BaseModel):
    # Message from chatbot
    role: Literal["system", "assistant", "user"]
    content: str   

    # parse the content into dictionary
    def dict(self) -> dict:
        current_key = None
        parsed = {}
        lines = self.content.splitlines()
        # parse the content line by line
        for line in lines:
            res = message_pattern.search(line)
            if res is not None:
                current_key = res.group(1)
                if res.group(2) is not None:
                    parsed[current_key] = res.group(2)
                else:
                    parsed[current_key] = ""
            elif current_key is not None:
                parsed[current_key] += "\n" + line
        return parsed

class Chapter(BaseModel):
    id: str
    next: str|None = None
    max_turns: int
    backgrounds: list[str]
    rules: list[str]
    initial: str
    
    def __init__(self, **kargs):
        super().__init__(**kargs)
    
    def get_messages(self) -> list[Message]:
        rendered_rules = render_rules(self.rules)
        system_content = Template(system).render({"rules": rendered_rules, "backgrounds":self.backgrounds, "character": character})
        initial_content = Template(self.initial).render({"rules": self.rules, "character": character})
        return [
            Message(role="system", content=system_content),
            Message(role="assistant", content=initial_content)
        ]

version = config["version"] # version of the config
system = config["system"] # system prompt template message
character = config["character"] # character information
chapters = config["chapters"] # chapters list
item_list = config["list"]["items"] # items list

messages:list[Message] =[] # chat messages

tags = [] # tags list
items = character["items"] # items list

print(f"Version: {version}, Chapters: {len(chapters)}")

# find the chapter by id
def getChapterById(id:str) -> Chapter:
    for r in chapters:
        if r["id"] == id:
            return Chapter(**r) 
    return None

# next chapter find and initialize
def choose_next_chapter():
    global current_chapter
    if current_chapter.next is not None:
        next = Template(current_chapter.next).render({"items": items, "tags": tags})
        print("Next round: ", next)
        next_chapter = getChapterById(next)
        if next_chapter is not None:
            current_chapter = next_chapter
            initials = current_chapter.get_messages()
            print(initials[1].content)
            show_controls(initials[1])
        else:
            print(f"Chapter '{next}' not found.")
    else:
        print("No next chapter found.")


# cout the messages by role, used for chapter's max_turns check
def count_messages(messages:list[Message], role:str) -> int:
    return len([msg for msg in messages if msg.role == role])

# chat with Azure OpenAI
def chat_with_azure(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = AzureOpenAI(api_key=os.environ.get("AZURE_OPENAI_API_KEY"), 
                         azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
                         api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
                         )
    stream = client.chat.completions.create(
        model=os.environ.get("AZURE_OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        max_tokens=4000,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if len(chunk.choices) > 0 and chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)
    
# chat with OpenAI
def chat(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        max_tokens=4000,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)

# render each rule from the chapter
def render_rules(rules:list[str]) -> str:
    lists = []
    for rule in rules:
        res = Template(rule).render({"character": character, "tags":tags, "items": items})
        if res is not None and res != "":
            lists.append(res)
    return lists

# do chat with the chatbot, and show the message with control buttons...
def do_chat(content:str|None, on_message: Union[callable, None] = None):
    if content is not None:
        turn = count_messages(messages, 'user') + 1
        if turn >= current_chapter.max_turns:
            content += f"\nwhisper: MAX_TURNS is reached."
        messages.append(Message(role="user", content=content))
        print(content)

    # change it to chat_with_azure to use Azure OpenAI
    concat = current_chapter.get_messages() + messages
    # for msg in concat:
    #     print(f"{msg.role}: {msg.content}")

    msg = chat_with_azure(concat, on_message)

    # print all the messages:
    show_controls(msg)
    messages.append(msg)
        
# handle player's action
def on_action(b, index:int):
    user_msg = f"I select '{index}'."
    do_chat(user_msg, lambda x: print(x, end=""))

# handle player's skill check
def on_skill(b, skill:str, difficulty:str):
    roll_num = roll("1d100").total
    if roll_num < 5:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL SUCCESS!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return
    if roll_num >= 95:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL FAILURE!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return

    difficulty_num = 15 if skill not in character["skills"] else character["skills"][skill]
    if difficulty == "hard":
        difficulty_num = int(difficulty_num * 0.5)
    if difficulty == "extreme":
        difficulty_num = int(difficulty_num * 0.2)
    
    print(f"rolled {roll_num} against {difficulty_num} on skill '{skill}'...")
    
    if roll_num <= difficulty_num:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a success!"
    else:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a failure."

    do_chat(user_msg, lambda x: print(x, end=""))

# handle continue
def on_continue(b):
    user_msg = f"Continue..."
    do_chat(user_msg, lambda x: print(x, end=""))

# handle chapter's ending, and show next chapter
def on_next(b):
    choose_next_chapter()
        
# handle player's retry, it will remove the last assistant message,
# and re-ask the chatbot
def on_retry(b):
    # delete last assistant message
    messages.pop()
    do_chat(None, lambda x: print(x, end=""))

# handle player's item usage
def on_item(b, item:dict):
    user_msg = Template(item["description"]).render({"character": character, "tags":tags, "items": items})
    user_msg = f"I use '{item['id']}' - {user_msg}"
    do_chat(user_msg, lambda x: print(x, end=""))

action_pattern = re.compile(r"(\d+). *(.+)")
# the main function to show the controls
def show_controls(msg:Message):
    dict = msg.dict()
    buttons = []

    if "choices" in dict.keys():
        # handle player's choices
        matches = action_pattern.finditer(dict["choices"])
        for match in matches:
            btn = widgets.Button(description=match.group(0), layout=widgets.Layout(width="fit-content"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        # handle skill check
        btn = widgets.Button(description=f"{dict['skill']} - {dict['difficulty']}", layout=widgets.Layout(width="fit-content"))
        btn.on_click(functools.partial(on_skill, skill=dict['skill'], difficulty=dict['difficulty']))
        buttons.append(btn)
    elif "ending" in dict.keys():
        # handle chapter ending, show next chapter button
        btn = widgets.Button(description=f"NEXT", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_next)
        buttons.append(btn)
    elif "game_over" in dict.keys():
        # handle game over
        print("\nGame Over - Hit 'Run All' button to restart again.")
    else:
        # If there is no choices or endings, show continue button
        btn = widgets.Button(description="Continue", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_continue)
        buttons.append(btn)
    
    # add tag from generated message
    if "add_tag" in dict.keys() and dict["add_tag"] not in tags:
        tags.append(dict["add_tag"])

    # remove tag from generated message
    if "remove_tag" in dict.keys() and dict["remove_tag"] in tags:
        tags.remove(dict["remove_tag"])
    
    # add item from generated message
    if "add_item" in dict.keys() and dict["add_item"] not in item_list:
        item = dict["add_item"].strip()
        found = next((x for x in item_list if x['id'] == item), None)
        if (found is None):
            print(f"\nItem '{dict['add_item'].strip()}' not found.")
        else:
            items.append(item)
    
    # remove item from generated message
    if "remove_item" in dict.keys() and dict["remove_item"] in items:
        items.remove(dict["remove_item"])
    
    controls = []

    # retry buttons
    if len(messages) > 0:
        retry_btn = widgets.Button(description="Retry", layout=widgets.Layout(width="fit-content"))
        retry_btn.on_click(on_retry)
        controls.append(retry_btn)

    # show tag list
    if len(tags) > 0:
        tags_lbl = widgets.Label(value=f"Tags: {', '.join(tags)}")
        controls.append(tags_lbl)

    # inventory buttons
    inventory = [widgets.Label(value="Inventory:")]
    if len(items) > 0:
        for item in items:
            found = next((x for x in item_list if x['id'] == item), None)
            if found:
                item_btn = widgets.Button(description=found["id"], layout=widgets.Layout(width="fit-content"))
                item_btn.on_click(functools.partial(on_item, item=found))
                inventory.append(item_btn)
            else:
                print(f"\nItem '{item}' not found in the list.")
        controls.append(widgets.HBox(inventory))
    
    display(widgets.VBox(buttons))

    if (len(controls) > 0):
        display(widgets.HBox(controls))

current_chapter = getChapterById("r1")

Version: 0.0.3, Chapters: 4


In [3]:
initials = current_chapter.get_messages()
print(initials[1].content)
show_controls(initials[1])

background: your_office
narration: 
You are a private detective. You used to be a police officer, but got fired because you found out a high-powered polician was the person behind a serial murder case.


You are hired by Mr. Knott to investigate his old Boston house. Weird things keep happening to the former tenants. A Macario kid went crazy and killed his parents. The Porter family got a strange disease and were committed to an asylum.

choices:
1. Go to the library, find historical reports.
2. Go to the police station, look into those cases.
3. Check the neighbourhood.


I select '3'.
background: neighbourhood
narration: 
You arrive in the neighborhood around the old Boston house. The streets are quiet, with only a few people milling about. You spot Mr. Dooley, the vendor of cigars and newspapers, whom you know might have some useful information.

skill: spot_hidden
difficulty: hard

rolled 43 against 35 on skill 'spot_hidden'...
i roll the dice for 'SPOT_HIDDEN' and get a failure.
narration: 
You try to focus and gather as much information as possible, but you find yourself distracted by the eeriness of the quiet streets. Mr. Dooley notices your presence but doesn't reveal anything significant, only general chatter about how the neighborhood has changed over the years.

choices:
1. Try to talk to Mr. Dooley again.
2. Head to the library to find historical reports.
3. Go to the police station to look into the cases.

I use 'alcohol' - A bottle of alcohol.
narration: 
You pull out a bottle of fine whiskey from your coat and offer it to Mr. Dooley. His eyes light up, and he gladly accepts, becoming more talkative. After a few shared swigs, he leans closer and begins to share some detailed information about the Macario and Porter families, confirming that entering through the back door of the house is safer.

add_tag: enter_back_door

choices:
1. Head to the library to find historical reports.
2. Go to the police station to look into the cases.
3. Approach the haunted house directly.

I select '3'.
background: neighbourhood
narration: 
You decide to head straight to the haunted house, emboldened by the information from Mr. Dooley about using the back door. The eerie silhouette of the house looms in the distance as you approach, shrouded in a foreboding mist.

ending: 
You stand in front of the haunted house, its windows dark and unwelcoming, the air filled with an unsettling silence. Your heart pounds with a mix of dread and determination as you prepare to enter and uncover the secrets that lie within.

Next round:  r2-0
background: front_door
narration: 
You're standing in front of the house. In spite of your experience, you still got shivered by what you have learnt about.

choices:
1. Go inside.
2. Find some tool nearby as your weapon, just in case.
